In [11]:
INPUT_BASE_DIR = "data/input"
OUTPUT_BASE_DIR = "data/output"

In [12]:
from pypdf import PdfReader

# creating a pdf reader object
reader = PdfReader(f'{INPUT_BASE_DIR}/31.05.2024.pdf')


# printing number of pages in pdf file
print(len(reader.pages))

# getting a specific page from the pdf file
page = reader.pages[0]

# # extracting text from page
# text = page.extract_text()
print(page)

1
{'/Contents': IndirectObject(15, 0, 4607984880), '/Type': '/Page', '/Resources': {'/ProcSet': ['/PDF', '/Text', '/ImageB', '/ImageC', '/ImageI'], '/Font': {'/F1': IndirectObject(4, 0, 4607984880), '/F2': IndirectObject(9, 0, 4607984880)}, '/XObject': {'/img9': IndirectObject(13, 0, 4607984880), '/img8': IndirectObject(12, 0, 4607984880), '/img7': IndirectObject(11, 0, 4607984880), '/img6': IndirectObject(10, 0, 4607984880), '/img5': IndirectObject(8, 0, 4607984880), '/img4': IndirectObject(7, 0, 4607984880), '/img3': IndirectObject(6, 0, 4607984880), '/img2': IndirectObject(5, 0, 4607984880), '/img1': IndirectObject(3, 0, 4607984880), '/img0': IndirectObject(2, 0, 4607984880)}}, '/Annots': [IndirectObject(14, 0, 4607984880)], '/Parent': IndirectObject(16, 0, 4607984880), '/MediaBox': [0, 0, 595.28, 841.88]}


In [13]:
import PyPDF2
import pandas as pd

# Define keywords to search for
keywords = ['Heizung', 'Warmwasser', 'Einheiten','aktueller Monat']  # Example keywords

# Function to check if a line contains any of the keywords
def contains_keyword(line, keywords):
    return any(keyword.lower() in line.lower() for keyword in keywords)

# Open the PDF file
pdf_path = f"{INPUT_BASE_DIR}/31.05.2024.pdf"
with open(pdf_path, "rb") as file:
    reader = PyPDF2.PdfReader(file)
    
    # Select the third page (index 2)
    page = reader.pages[0]
    
    # Extract text from the page
    text = page.extract_text()
    
    # Split the text into lines
    lines = text.split('\n') if text else []

    # Filter lines based on keywords
    filtered_lines = [line for line in lines if contains_keyword(line, keywords)]

# Prepare data for DataFrame
text_data = [{'line': line} for line in filtered_lines]

# Create DataFrame
df_filtered = pd.DataFrame(text_data)

# Save DataFrame to CSV
df_filtered.to_csv(f"{OUTPUT_BASE_DIR}/filtered_lines.csv", index=False)

print("Filtered data saved to CSV file.")


Filtered data saved to CSV file.


In [14]:
import pdfplumber
import pandas as pd

# Define keywords to search for
keywords = ['Verbrauch im','Heizung', 'Warmwasser', 'Einheiten','aktueller Monat']  # Example keywords

# Function to check if a line contains any of the keywords
def contains_keyword(line, keywords):
    return any(keyword.lower() in line.lower() for keyword in keywords)

# Open the PDF file
pdf_path = f"{INPUT_BASE_DIR}/31.05.2024.pdf"
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]
    
    # Extract text and bounding boxes
    text_data = []
    for word in page.extract_words():
        text = word['text']
        if contains_keyword(text, keywords):
            text_data.append({
                'text': text,
                'x0': word['x0'],
                'y0': word['top'],
                'x1': word['x1'],
                'y1': word['bottom']
            })

    # Create DataFrame
    df_filtered = pd.DataFrame(text_data)
    
    # Save DataFrame to CSV
    df_filtered.to_csv(f"{OUTPUT_BASE_DIR}/filtered_rectangles.csv", index=False)

print("Filtered rectangles saved to CSV file.")


Filtered rectangles saved to CSV file.


In [15]:
df_filtered

,text,x0,y0,x1,y1
0,Heizung,89.440,480.485,133.19125,491.735
1,Einheiten,202.385,498.730,233.64500,506.230
2,Einheiten,263.165,498.730,294.42500,506.230
3,Einheiten,331.755,498.730,363.01500,506.230
4,Warmwasser,89.440,530.215,160.07875,541.465


In [16]:
rect = (50, 430, 500, 560)  # Example coordinates

# Open the PDF file
pdf_path = f"{INPUT_BASE_DIR}/31.05.2024.pdf"
with pdfplumber.open(pdf_path) as pdf:
    # Select the page
    page = pdf.pages[0]  # Page index starts at 0

    # Crop to the rectangular area
    cropped_page = page.within_bbox(rect)
    
    # Extract text from the cropped area
    text = cropped_page.extract_text()

print("Extracted Text:")
print(text)

Extracted Text:
Verbrauch im Verbrauch im April Verbrauch im Mai aktueller Monat im Vergleich
Mai 2024 2024 2023 anderen Haushalten
Heizung 0 kWh 4 kWh 0 kWh 100 % weniger verbrau
0 Einheiten 3 Einheiten 0 Einheiten
Warmwasser 172 kWh 61 kWh 133 kWh 45 % mehr verbraucht
1,7 m³ 0,6 m³ 1,5 m³


In [17]:
import tabula

# tabula.environment_info()


In [18]:
dfs = tabula.read_pdf(f'{INPUT_BASE_DIR}/31.05.2024.pdf', stream=True)


'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Sep 04, 2024 11:59:17 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 04, 2024 11:59:17 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 04, 2024 11:59:17 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 04, 2024 11:59:17 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

